# Práctica 1 — Dataset 2: Renta mediana por Census Tract (Bronze ➜ Silver)

**Objetivo:** Leer el CSV de ingresos desde **bronze**, filtrar únicamente los **Census Tracts** de San Francisco,
limpiar/transformar (incluyendo censura `250,000+`) y persistir en **silver** como **Parquet**.

## Requisitos previos (manual)

1. Descarga desde **Moodle**: `acs_us_median_household_income_2019-2023.zip`
2. Descomprime y **sube solo** este fichero a MinIO (bucket **bronze**), en la carpeta/prefijo:

   - `sf_median_household_income/ACSST5Y2023.S1901-Data.csv`

   (Los otros ficheros del ZIP son metadatos y NO deben cargarse a bronze para esta práctica.)

In [ ]:
# ============================================================
# 0) Imports + SparkSession
# ============================================================
from pyspark.sql import SparkSession, functions as F, types as T
import os

spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "UTC")

print("Spark version:", spark.version)

In [ ]:
# ============================================================
# 0.1) Configuración MinIO (S3A)
# ============================================================
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT", "http://minio:9000")
MINIO_ACCESS_KEY = os.getenv("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.getenv("MINIO_SECRET_KEY", "minioadmin")
MINIO_USE_SSL = os.getenv("MINIO_USE_SSL", "false").lower() == "true"

hconf = spark._jsc.hadoopConfiguration()
hconf.set("fs.s3a.endpoint", MINIO_ENDPOINT)
hconf.set("fs.s3a.access.key", MINIO_ACCESS_KEY)
hconf.set("fs.s3a.secret.key", MINIO_SECRET_KEY)
hconf.set("fs.s3a.path.style.access", "true")
hconf.set("fs.s3a.connection.ssl.enabled", "true" if MINIO_USE_SSL else "false")
hconf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
hconf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

print("MINIO_ENDPOINT =", MINIO_ENDPOINT)

## 1) Lectura desde Bronze (sin inferir esquema)

Se lee **a nivel de carpeta**.

In [ ]:
BRONZE_PATH = "s3a://bronze/sf_median_household_income/"

df_raw = (
    spark.read
        .option("header", True)
        .option("inferSchema", "false")
        .csv(BRONZE_PATH)
)

df_raw.printSchema()
df_raw.show(5, truncate=False)

## 2) Filtrar filas de interés (solo Census Tracts de San Francisco)

El enunciado pide quedarnos con las filas cuyo `GEO_ID` empiece por:

- `1400000US`

(es decir, el nivel de *Census Tract*).

In [ ]:
df_filtered = df_raw.filter(F.col("GEO_ID").startswith("1400000US"))

print("Filas tras filtrado:", df_filtered.count())
df_filtered.select("GEO_ID", "NAME").show(5, truncate=False)

## 3) Crear columna `census_tract` (string) extrayendo la parte posterior a `US`

Ejemplo: `1400000US06075010101` → `06075010101`

In [ ]:
df_with_tract = df_filtered.withColumn(
    "census_tract",
    F.regexp_extract(F.col("GEO_ID"), r"US(\d+)$", 1)
)

df_with_tract.select("GEO_ID", "census_tract").show(10, truncate=False)

## 4) Seleccionar columnas y renombrar

Columnas requeridas:
- `census_tract`
- `S1901_C01_012E` → `median_household_income_raw`
- `NAME` → `tract_name`

In [ ]:
df_sel = df_with_tract.select(
    F.col("census_tract"),
    F.col("S1901_C01_012E").alias("median_household_income_raw"),
    F.col("NAME").alias("tract_name")
)

df_sel.printSchema()
df_sel.show(10, truncate=False)

## 5) Crear `median_household_income` (double) a partir de `median_household_income_raw`

- Hay valores especiales como `250,000+` que deben convertirse a `250000`.
- Además, eliminamos separadores `,` y cualquier carácter no numérico.

In [ ]:
# Nos quedamos con los dígitos (esto convierte "250,000+" -> "250000")
income_digits = F.regexp_replace(F.col("median_household_income_raw"), r"[^0-9]", "")

df_typed = df_sel.withColumn(
    "median_household_income",
    F.when(income_digits == "", F.lit(None).cast("double"))
     .otherwise(income_digits.cast("double"))
)

df_typed.select("median_household_income_raw", "median_household_income").show(20, truncate=False)

## 6) Crear `income_is_censored` (boolean)

- `true` cuando `median_household_income_raw` sea exactamente `250,000+`
- `false` en cualquier otro caso

In [ ]:
df_final = df_typed.withColumn(
    "income_is_censored",
    (F.col("median_household_income_raw") == F.lit("250,000+")).cast("boolean")
)

df_final.printSchema()
df_final.show(10, truncate=False)

## 7) Escritura en Silver (Parquet)

Persistimos el DataFrame final en:

- `silver/sf_median_household_income/`

In [ ]:
SILVER_PATH = "s3a://silver/sf_median_household_income/"

(
    df_final.write
      .mode("overwrite")
      .parquet(SILVER_PATH)
)

print("Escritura completada en:", SILVER_PATH)

# (Opcional) Lectura de verificación
df_check = spark.read.parquet(SILVER_PATH)
print("Filas en silver:", df_check.count())
df_check.printSchema()

## 8) Preguntas analíticas

### i) ¿Cuántos census tracts hay en total tras el filtrado?

In [ ]:
total_tracts_rows = df_final.count()
total_tracts_distinct = df_final.select("census_tract").distinct().count()

print("Filas (tracts) tras filtrado:", total_tracts_rows)
print("Census tracts distintos:", total_tracts_distinct)

**Respuesta (completa tras ejecutar):**  
- **Resultado:** `TODO`  
- **Nota:** si hubiera duplicados inesperados, la métrica “tracts distintos” es la más robusta.

### ii) ¿Cuántos census tracts no tienen información de renta (`median_household_income` nulo)?

In [ ]:
tracts_income_null = df_final.filter(F.col("median_household_income").isNull()).count()
print("Census tracts con median_household_income nulo:", tracts_income_null)

**Respuesta (completa tras ejecutar):**  
- **Resultado:** `TODO`  
- **Comentario:** estos tracts no aportan información para análisis de renta; según el caso, se pueden excluir o tratar aparte.

### iii) ¿Cuántos tracts tienen la renta censurada (`income_is_censored = true`)?

In [ ]:
tracts_censored = df_final.filter(F.col("income_is_censored") == True).count()
print("Census tracts con renta censurada:", tracts_censored)

**Respuesta (completa tras ejecutar):**  
- **Resultado:** `TODO`  
- **Comentario:** la censura indica que el valor real es **>= 250000**, pero no se conoce el exacto; ojo con medias y modelos.

### iv) ¿Cuál es el mínimo, máximo y media de `median_household_income` (ignorando nulos)?

In [ ]:
stats = (
    df_final
    .select(
        F.min("median_household_income").alias("min_income"),
        F.max("median_household_income").alias("max_income"),
        F.avg("median_household_income").alias("avg_income"),
    )
    .collect()[0]
)

print("min_income:", stats["min_income"])
print("max_income:", stats["max_income"])
print("avg_income:", stats["avg_income"])

**Respuesta (completa tras ejecutar):**  
- **Mínimo:** `TODO`  
- **Máximo:** `TODO`  
- **Media:** `TODO`  
- **Comentario:** la media puede verse afectada por la censura en 250k+ (hemos fijado 250000 como valor mínimo para esos casos).